In [3]:
import pandas as pd
import numpy as np


def load_and_process(path):

        # Method Chain 1, load data and drop and replace columns

    df = (pd.read_csv(path,sep=',')
          .drop(['Job Description',"Easy Apply","Competitors","Industry","Sector","Revenue","Type of ownership","Headquarters","Founded"],axis='columns') 
          .rename(columns = {"Rating" : "Company Rating","Size":"Company Size"})
         )

        # Method Chain 2   
        # Replace unknown data in Company Rating and Size Column 

    df1 = (pd.DataFrame(data=df)
           .loc[lambda x: x["Company Rating"]==-1]
           .replace(-1,0)
          )
    df.update(df1)

    df1 = (pd.DataFrame(data=df)
           .loc[lambda x: x["Company Size"]=="-1"]
           .replace("-1","Unknown")
          )
    df.update(df1)


        # Method Chain 3, Remove new line("\n") in Company name column
        # Remove (Glassdoor est.) in Salary Estimate column and create new Min and Max column

    df2 = (pd.DataFrame(data=df["Company Name"])
          .replace(r'\n.*', '', regex=True)
         )
    df.update(df2)

    df2 = (pd.DataFrame(data=df["Salary Estimate"])
          .replace(r'\(.*', '', regex=True)
          .replace('[\$]', '', regex=True)
         )    
    df.update(df2)

    new = df["Salary Estimate"].str.split("-", n = 1, expand = True)
    df["Minimum Salary"]= new[0]
    df["Maximum Salary"]= new[1]
    df.drop(columns =["Salary Estimate"],inplace = True)

    df2 = (pd.DataFrame(data=df["Minimum Salary"])
          .replace('K', '', regex=True)
          .astype(int)
         )
    df2=df2*1000
    df.update(df2)

    df2 = (pd.DataFrame(data=df["Maximum Salary"])
          .replace('K', '', regex=True)
          .astype(int)
         )
    df2=df2*1000
    df.update(df2)
    df["Average Salary"] = (df["Maximum Salary"]+df["Minimum Salary"])/2


        # Cut down data size to only include Data Engineer, Software Engineer, and Big Data Engineer job title

    DataEng_data = df.loc[df["Job Title"]== "Data Engineer"] #469 rows
    SoftEng_data = df.loc[df["Job Title"]== "Software Engineer"] #93 rows
    BigDataEng_data = df.loc[df["Job Title"]== "Big Data Engineer"] #73 rows
    frames = [DataEng_data,SoftEng_data,BigDataEng_data]
    df = pd.concat(frames)
    df = df.reset_index()


        # Caterogized company size column

    df["Company Size"].loc[df["Company Size"] == "1 to 50 employees"] = "Small"
    df["Company Size"].loc[df["Company Size"] == "51 to 200 employees"] = "Small"
    df["Company Size"].loc[df["Company Size"] == "201 to 500 employees"] = "Medium"
    df["Company Size"].loc[df["Company Size"] == "501 to 1000 employees"] = "Large"
    df["Company Size"].loc[df["Company Size"] == "1001 to 5000 employees"] = "Large"
    df["Company Size"].loc[df["Company Size"] == "5001 to 10000 employees"] = "Large"
    df["Company Size"].loc[df["Company Size"] == "10000+ employees"] = "Large"
    
    return df

                                                          

In [4]:
load_and_process('data/raw/DataEngineer.csv')

C:\Users\myste\miniconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,index,Job Title,Company Rating,Company Name,Location,Company Size,Minimum Salary,Maximum Salary,Average Salary
0,0,Data Engineer,4.5,Sagence,"New York, NY",Small,80000,150000,115000.0
1,4,Data Engineer,3.7,GNY Insurance Companies,"New York, NY",Medium,80000,150000,115000.0
2,7,Data Engineer,4.2,NJF Global Holdings,"New York, NY",Small,80000,150000,115000.0
3,15,Data Engineer,3.8,Citadel,"New York, NY",Large,80000,150000,115000.0
4,16,Data Engineer,4.4,PulsePoint,"New York, NY",Small,80000,150000,115000.0
...,...,...,...,...,...,...,...,...,...
630,2366,Big Data Engineer,3.7,Tata Consultancy Services (North America),"Irving, TX",Large,60000,109000,84500.0
631,2373,Big Data Engineer,4.4,Tachyon Technologies LLC,"Irving, TX",Medium,60000,109000,84500.0
632,2375,Big Data Engineer,0.0,Node.Digital LLC,"Arlington, TX",Small,60000,109000,84500.0
633,2378,Big Data Engineer,3.8,Enterprise Solutions Inc,"Irving, TX",Small,60000,109000,84500.0
